# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
This is a Classification problem.
This project can be efficiently solved by classifying students into two discrete groups, Group 1 who don't need intervention and Group 2 those who need intervention.

Regression on the other hand is best fit for predicting continious outputs.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [54]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
print student_data
#print len(student_data)


Student data read successfully!
    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
5       GP   M   16       U     LE3       T     4     3  services     other   
6       GP   M   16       U     LE3       T     2     2     other     other   
7       GP   F   17       U     GT3       A     4     4     other   teacher   
8       GP   M   15       U     LE3       A     3     2  services     other   
9       GP   M   15       U     GT3       T     3     4     other     other   
10      GP   F   15       U     GT3       T     4     4   teacher    health   
11      GP   F   15 

### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [52]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = (np.size(student_data)/n_students)-1

# TODO: Calculate passing students
c=0
for _, p in student_data.iterrows():
    if p['passed']=='yes':
        c=c+1
#print c
n_passed = c

# TODO: Calculate failing students
n_failed = n_students-c

# TODO: Calculate graduation rate
grad_rate = float(n_passed)*100/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [9]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import ShuffleSplit
from sklearn.cross_validation import train_test_split 

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

#print num_test

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.24, random_state=20)

X_train = X_train
X_test = X_test
y_train = y_train
y_test = y_test

new= X_train.head(100)
print len(new)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

100
Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
1. Support Vector Machines (SVM)

Real world application:
a. SVM can be used for Gender Classification Based on 3D Face Geometry 
http://ieeexplore.ieee.org/document/5279671/
 Face Detection with Facial Features and Gender Classification
http://www.itfrindia.org/ICCIC/Vol2/9018ICCIC.pdf

b. Recognizing hand-written digits
http://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html

Strengths:
SVM supports kernels & soft-margin SVM, so we can model even non-linear relations.
Training is relatively easy. No local optimal, unlike in neural networks.
It scales relatively well to high dimensional data.

Weakness:
One limitation of the support vector approach lies in choice of the kernel. Incorrect choice of kernel can lead to overfitting or high-bias.
A second limitation is speed and size, both in training and testing.


SVM is a classification SLM which seperates data into different groups by a dividing the hyperplane. In this project our goal is to classify students dataset into two, students who need intervention and students who dont. Thus SVM can be used to find a hyperplane to seperate students into above two groups as required.
Also natural task for SVM is binary classification, which is exactly the case in this project.Also it scales relatively well to high dimensional data (given there are 30 features). Also as data is not large, speed and size will not be an issue.


2. Decision Tree:
Real world application:
a. Object recognition: Tree based classification has been used recently for recognizing three dimensional objects [338,39] and for high level vision [184].
http://www.cbcb.umd.edu/~salzberg/docs/murthy_thesis/survey/node32.html

b. Molecular biology: Decision trees for analyzing amino acid sequences.
http://www.cbcb.umd.edu/~salzberg/docs/murthy_thesis/survey/node32.html

Strengths:
1. Decision trees implicitly perform variable screening or feature selection.
When we fit a decision tree to a training dataset, the top few nodes on which the tree is split are essentially the most important variables within the dataset and with decision tree feature selection is completed automatically.
2. Decision trees require relatively little effort from users for data preparation.
o overcome scale differences between parameters - for example if we have a dataset which measures revenue in millions and loan age in years, say; this will require some form of normalization or scaling before we can fit a regression model and interpret the coefficients.  Such variable transformations are not required with  decision trees because the tree structure will remain the same with or without the transformation.

Another feature which saves data prep time: missing values will not prevent splitting the data for building trees. 
Decision trees are also not sensitive to outliers since the splitting happens based on proportion of samples within the split ranges and not on absolute values.

3. Nonlinear relationships between parameters do not affect tree performance.
Decision trees do not require any assumptions of linearity in the data. Thus, we can use them in scenarios where we know the parameters are nonlinearly related.

4. The best feature of using trees for analytics - easy to interpret and explain to executives!

Weakness:
1. Without proper pruning or limiting tree growth, they tend to overfit the training data, making them somewhat poor predictors.
2. 	Tree splitting is locally greedy – At each level, tree looks for binary split such that impurity of tree is reduced by maximum amount. This is a greedy algorithm and may result in local optima.


3. Ensemble Methods(Boosting)
Real world applications:
a. Estimation of Osteoporotic Fracture Cases by Using Ensemble Learning Approaches.
https://www.ncbi.nlm.nih.gov/pubmed/26660692

b. Ensemble learning approaches used in the identification of disease mutations
http://ieeexplore.ieee.org/abstract/document/5639753/

Strengths:
1. Does not learn a single classifier but learns a set of classifiers & combines the predictions of multiple classifiers, parameters to get higher accuracy.
2. Reduces overfitting.


Weakness:
1. Increased storage. Instead of a single classifier all component classifiers,need to be stored after training. The total storage depends on the size of each component classifier itself and the size of the ensemble (number of classifiers in the ensemble).Performance will be limited when computational storage is limited.
2. Increased computation. Takes longer time,(* at times days) to provide results since  to classify an input query, all component classifiers (instead of a single classifier) must be processed, and thus it requires more execution time.  Not suitable when smaller computation time is a factor.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [10]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [16]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import neighbors
from sklearn import svm
from sklearn import linear_model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# TODO: Initialize the three models
#clf_A = svm.SVC()
#clf_A = linear_model.SGDClassifier()
#clf_A = AdaBoostClassifier()
#clf_A = GaussianNB()
#clf_A= BaggingClassifier()
clf_A= RandomForestClassifier()
#clf_A= svm.SVC()
clf_B = DecisionTreeClassifier()
clf_C = GradientBoostingClassifier()

# TODO: Set up the training set sizes
X_train_100 = X_train.head(100)
y_train_100 = y_train.head(100)

X_train_200 = X_train.head(200)
y_train_200 = y_train.head(200)

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
train_predict( clf_A , X_train_300, y_train_300, X_test, y_test)

Training a RandomForestClassifier using a training set size of 300. . .
Trained model in 0.0100 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.9950.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7746.


### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SVM**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.0020                 |  0.0010                | 0.8889           |  0.8077         |
| 200               |  0.0040                 |  0.0030                | 0.8669           |  0.8205         |
| 300               |  0.0090                 |  0.0060                | 0.8565           |  0.8205         |

** Classifer 2 - DecisionTree Classifier

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0010              |   0.0020               |  1.0000          | 0.7077          |
| 200               |     0.0020              |   0.0030               |  1.0000          | 0.6615          |
| 300               |     0.0020              |   0.0000               |  1.0000          | 0.7883          |

** Classifer 3 - Ensemble Methods (GradientBoostingClassifier)

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.0680               |   0.0000               |  1.0000          | 0.7612         |
| 200               |    0.1020               |   0.0010               |  0.9858          | 0.8299          |
| 300               |    0.1480               |   0.0010               |  0.9592          | 0.8108          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
F1 score, cost & time performance are some of the key features which need to be considered while selecting a ML model.

1. For the given problem, the amount of data is fairly small. So performance time (fitting, prediction) of any ML model will be fairly small (less than 1 second). So time performance should not be a factor to decide a model in this case.
2. Cost: cost can be determined in terms of time it takes to run as well as storage/memory it requires. Again these factors will only play a role if the dataset is huge(in Terabytes range). Since dataset in this problem is significantly small to make any impact of storage requirements based on model selected, the cost of operation will be fairly equal for any ML model.
3. Given above two, the only key feature to look while selecting a model will be how accurate it is while predicting new data. Model with best F1 score should be selected.

SVM, DecisionTree and Ensemble Methods (GradientBoostingClassifier) were used to perform trials on the provided dataset.
We observed, all three models had almost similar testing F1 Scores.(withing .0322 of each other range(0.7883- .8205)).
However Ensemble Methods (GradientBoostingClassifier) performed considerably well in fitting the training dataset with F1 score of 0.9592 and prediting testing data with a F1 score of 0.8108.
This shows that although the Ensemble Methods (GradientBoostingClassifier) is overfitting in comparision to SVM, it still is acheiving almost equivalent F1 test scores to SVM.
So by fine tuning parameters using GridsearchCV, Ensemble Methods (GradientBoostingClassifier) should be able to bring it's F1 test score closer to its training F1 score which is almost 1.

So Ensemble Methods (GradientBoostingClassifier) should be an appropriate model for this project.



### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: 
In this project, we have 395 number of students, each having 30 features such as school, sex,  age, address famsize, interests etc which the alogrithm will use to classfiy students with a Pass or Fail label. Of the 395 students datapoint, we split 300 data points to train the ML model. The rest 95 data points were then used to verify how well the ML model was able to predict whether a student Passed or Fail by comparing ML output with the actual pass/fail results. The ML Algorithm which performed best was chosen.
Based on the prediction performance Ensemble Methods (GradientBoostingClassifier) ML algorithm was chosen. 
Basic idea:
Ensemble methods combine learnings from multiple learning algorithms to obtain a better predictive performance rather than just using any one algorithm alone.
Ensemble Methods(GradientBoostingClassifer) algorithm produced a more accurate prediction model by combining information gained from multiple weak/not as accurate base prediction models.Base prediction model used was decision trees.
Often the ML model performance can be enhanced by varying the default parameters of the model as per the problem at hand.Thus further performance increase was achieved by using GridSearchCV program which creates a new best model by varying the defaul parameters for GradientBoostingClassifer.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [44]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn.metrics import make_scorer, fbeta_score, f1_score

# TODO: Create the parameters list you wish to tune
parameters = {'n_estimators':[100,50,75,200],'learning_rate':[1,.5,2],'max_depth':[3,5,7],'random_state':[0,5]}

#parameters = {'n_estimators':[48, 40],'max_depth':[3,5,7],'random_state':[0,5],'min_samples_split':[5,2]}

# TODO: Initialize the classifier
clf = GradientBoostingClassifier()
#clf = RandomForestClassifier()
# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer =  make_scorer(f1_score,pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(clf, parameters)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0000 seconds.
Tuned model has a training F1 score of 1.0000.
Made predictions in 0.0000 seconds.
Tuned model has a testing F1 score of 0.8552.


#### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: 
Final Model's F1 score before tuning:

Training a GradientBoostingClassifier using a training set size of 300. . .
Trained model in 0.1190 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.9592.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.8108.

Final Model's F1 score after tuning:
Training a GradientBoostingClassifier using a training set size of 300. . .
Made predictions in 0.0010 seconds.
Tuned model has a training F1 score of 1.0000.
Made predictions in 0.0000 seconds.
Tuned model has a testing F1 score of 0.8552


As can be observed after tuning the training F1 score has improved from .9592 to 1.0 by 0.0408(4.25%)
Testing F1 score has improved from 0.8108 to 0.8552 by 0.0444(5.47%)


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.